##### Complaint Agent Stream

This notebook creates a scheduled job to process complaints through the complaint agent

In [ ]:
%pip install --upgrade databricks-sdk

In [ ]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.jobs as j
import os

w = WorkspaceClient()

# Resolve the workspace notebook path
notebook_abs_path = os.path.abspath("../jobs/complaint_agent_stream")
notebook_dbx_path = notebook_abs_path.replace(
    os.environ.get("DATABRICKS_WORKSPACE_ROOT", "/Workspace"),
    "/Workspace"
)

job = w.jobs.create(
    name="Complaint Agent Stream",
    schedule=j.CronSchedule(
        quartz_cron_expression="0 0/10 * * * ?",  # Every 10 minutes
        timezone_id="UTC",
        pause_status=j.PauseStatus.UNPAUSED),
    tasks=[
        j.Task(
            task_key="complaint_agent_stream",
            notebook_task=j.NotebookTask(
                notebook_path=notebook_dbx_path,
                base_parameters={
                    "CATALOG": dbutils.widgets.get("CATALOG"),
                    "COMPLAINT_AGENT_ENDPOINT_NAME": dbutils.widgets.get("COMPLAINT_AGENT_ENDPOINT_NAME")
                },
            )
        )
    ],
)
print(f"Created job_id={job.job_id}")

import sys
sys.path.append('../utils')
from uc_state import add

add(dbutils.widgets.get("CATALOG"), "jobs", job)

w.jobs.run_now(job_id=job.job_id)